In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers, utils, losses

In [2]:
# colab on google drive 

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/groom/semi-3/fifa_pay_prediction/03_work_code/ukseongchoi'

Mounted at /content/drive
/content/drive/MyDrive/groom/semi-3/fifa_pay_prediction/03_work_code/ukseongchoi


In [3]:
train = pd.read_csv("../../00_original_data/FIFA_train.csv")
test = pd.read_csv("../../00_original_data/FIFA_test.csv")

submission = pd.read_csv("../../00_original_data/submission.csv")


In [4]:
# 필요하지 않는 feature 값 제거
train.drop(['id', 'name'], axis = 1, inplace = True)
test.drop(['id', 'name'], axis = 1, inplace = True)

In [5]:
#타겟 데이터
y_train = train['value']

#타겟 생성 후 필요없는 열제거
del train['value']

#학습 데이터
x_train = train.copy()

In [6]:
print(x_train['continent'].unique())
print(x_train['position'].unique())
print(x_train['prefer_foot'].unique())

['south america' 'europe' 'africa' 'asia' 'oceania']
['ST' 'GK' 'DF' 'MF']
['left' 'right']


In [7]:
# categorical data를 담고있는 Series는 특수 메서드인 cat 속성이 있다.
# 이를 통해 catagories 와 codes 나 categorical 메서드 등에 쉽게 접근할 수 있다.
# 하지만 시리즈에 바로 categories나 codes를 사용하면 에러가 발생하는데 이때 이용하는 것이 cat 속성이다.

x_train['continent']=x_train['continent'].astype('category').cat.codes
test['continent'] = test['continent'].astype('category').cat.codes

x_train['position'] = x_train['position'].astype('category').cat.codes
test['position'] = test['position'].astype('category').cat.codes

x_train['prefer_foot'] = x_train['prefer_foot'].astype('category').cat.codes
test['prefer_foot'] = test['prefer_foot'].astype('category').cat.codes


In [8]:
# 계약 연도를 object -> int 로 변환
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:] # 계약연도가 제일 끝에 있거나 단독으로 쓰이고 있어 뒤에서 4번째부터 가지고 옴.
    return int(string)

x_train['contract_until'] = x_train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

In [203]:
print(y_train)
x_train
# test

0       110500000.0
1        72000000.0
2        80000000.0
3        51000000.0
4        68000000.0
           ...     
8927        60000.0
8928        40000.0
8929        50000.0
8930        50000.0
8931        60000.0
Name: value, Length: 8932, dtype: float64


,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,31,4,2021,3,0,5.0,94,94,4.0
1,27,2,2020,1,1,4.0,91,93,1.0
2,31,4,2021,3,1,5.0,91,91,3.0
3,32,2,2020,0,1,4.0,91,91,3.0
4,25,2,2021,1,1,3.0,90,93,1.0
...,...,...,...,...,...,...,...,...,...
8927,18,0,2019,2,1,1.0,48,63,3.0
8928,19,2,2020,0,1,1.0,47,59,2.0
8929,18,4,2021,0,1,1.0,47,64,2.0
8930,18,2,2021,1,1,1.0,47,65,1.0


In [114]:
#조합찾기 

from itertools import combinations 

arr = ['reputation' , 'stat_skill_moves']
categorics = []
for i in range(len(arr)+1):
  categorics = categorics + list(combinations(arr,i))

for k in range(len(categorics)):
  categorics[k] = list(categorics[k])

numerics = []
for i in range(len(categorics)):
  remove_set = list(categorics[i])
  numerics.append([i for i in arr if i not in remove_set])

categorical_base = ['continent', 'position']
numerical_base = ['stat_overall', 'stat_potential','contract_until', 'age']

for k in range(len(categorics)):
  categorics[k] = categorical_base + categorics[k]
  numerics[k] = numerical_base + numerics[k]

[['continent', 'position'], ['continent', 'position', 'reputation'], ['continent', 'position', 'stat_skill_moves'], ['continent', 'position', 'reputation', 'stat_skill_moves']] [['stat_overall', 'stat_potential', 'contract_until', 'age', 'reputation', 'stat_skill_moves'], ['stat_overall', 'stat_potential', 'contract_until', 'age', 'stat_skill_moves'], ['stat_overall', 'stat_potential', 'contract_until', 'age', 'reputation'], ['stat_overall', 'stat_potential', 'contract_until', 'age']]


In [250]:
new_df = x_train.copy()
pd_list = []
for j in range(len(categorics)):
  new_df = x_train.copy()

  for categori_name in categorics[j]:


    num = len(new_df[categori_name].unique())
    make_name =[]
    for i in range(num):
      
      make_name.append(categori_name + str(i))

    en_df = pd.get_dummies(new_df[categori_name])

    en_df.columns = make_name


    del new_df[categori_name]

    new_df = pd.concat([new_df , en_df], axis=1 )

  pd_list.append(new_df)

In [257]:
#pd_list[0]